# IFT3700 A2023 - TP

# Introduction

Dans le cadre de notre travail final pour le cours IFT3700 A2023, notre équipe composée de [TODO: Noms des Membres] a entrepris la tâche de collecter, nettoyer et analyser des données numériques et catégoriques relatives à divers pays du monde. Ces données ont été extraites de 40 tableaux provenant de sources fiables sur Wikipédia (disponible dans l'énoncé du TP). Notre objectif était de nettoyer ces données, de les discrétiser en format binaire et de mener une analyse approfondie pour en extraire des insights pertinents.

Notez que certains imports sont nécessaires

# Méthodologie

## Collecte des données

Nous avons collecté les données à partir des sources spécifiées dans l'énoncé. Nous avons utilisé des scripts afin de mener à bien le webscraping. Pour tester notre code, voici un script qui effectue le scraping un url à la fois.

On commence par importer les librairies nécessaires

In [105]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import unquote #proposed by chatgpt to get url content name as a string

URL wikipedia:

In [106]:
url="https://en.wikipedia.org/wiki/List_of_countries_with_highest_military_expenditures"

On envoie un request à l'URL pour parse le contenue

In [107]:
response=requests.get(url)
soup=BeautifulSoup(response.content, 'html.parser')

On trouve la table dans le contenu HTML

In [108]:
tables = soup.find_all('table', {'class': 'wikitable'})

Extraction proposé par ChatGPT afin d'extraire le nom de l'article wikipedia

In [109]:
page_title = unquote(url.split('/')[-1])

On génère des fichiers CSV à partir du contenu DataFrame (un fichier si une seule table est présente)

In [110]:
# Initialize an empty list to store DataFrames
dfs = []

# Initialize an empty list to store DataFrames for each table on the current page
dfs_per_page = []

# Iterate through the list of tables and convert each to a DataFrame
for i, table in enumerate(tables):
    df = pd.read_html(str(table), header=0)[0]  # Assuming headers are in the first row
    dfs_per_page.append(df)

    # Write each DataFrame to a separate CSV file
    filename = 'table_test_1.'+ str(i+1) + "_" + page_title + '.csv'
    with open(filename, 'w', newline='') as file:
        df.to_csv(file, index=False)

Les fichiers CSV sont créés. (1 fichier si une seule table présente)

Pour faire une extraction de tous les liens disponible dans le google doc, nous avons un script qui le fait disponible au lien suivant : 
https://github.com/mildshield14/ift3700_tp/blob/main/webscraping/script_extractdata.py

## Nettoyage des Données

### Gestion des valeurs manquantes

Nous avons .....

In [111]:
#TODO

### Suppression des Duplicatas

Nous avons....

In [112]:
#TODO

### Gestion des outliers

Nous avons ...

In [113]:
#TODO 

### Discrétisation des Caractéristiques


Nous avons ....

In [114]:
#TODO

## Analyse des Données

### Calcul des statistiques descriptives

Pour chaque ....

In [115]:
#TODO

### Calcul des Corrélations

Calcul coeff de corr etc

In [116]:
#TODO

### Prédictions

Ref lin etc

In [117]:
#TODO

### Visualisation et Représentations

Nous avons ...

In [118]:
#TODO

# Résultats

## Statistiques descriptives

[TODO : Inclure mean median etc]

## Corrélations

[TODO]

## Prédictions

[TODO : Regr Lin, class bayes etc ]

## Visualisation

[TODO- Inclure images]

# Conclusion

Dans ce trvail, ...